In [1]:
import os
import json
import logging
import asyncio
import aiohttp
import spacy
import yfinance as yf
import structlog
import numpy as np
import nest_asyncio
from datetime import datetime
from bs4 import BeautifulSoup
from openai import OpenAI
from textblob import TextBlob
from tavily import TavilyClient
from redis import Redis
from pydantic_settings import BaseSettings
from typing import Optional, Dict, List, Any

logging.basicConfig(level=logging.DEBUG)

nest_asyncio.apply()

class Settings(BaseSettings):
    GROQ_API_KEY: str
    TAVILY_API_KEY: str
    HF_API_TOKEN: str
    REDIS_URL: str
    DEBUG_LEVEL: str = "DEBUG"
    CACHE_TTL: int = 3600

    class Config:
        env_file = ".env"

class MarketAnalyzer:
    def __init__(self):
        self.config = Settings()
        self.nlp = spacy.load("en_core_web_sm")
        self.redis = Redis.from_url(self.config.REDIS_URL)
        self.logger = structlog.get_logger()
        self.tavily_client = TavilyClient(api_key=self.config.TAVILY_API_KEY)
        self.openai_client = OpenAI(
            base_url="https://api.groq.com/openai/v1",
            api_key=self.config.GROQ_API_KEY,
            max_retries=5,
            timeout=30
        )
        
    async def get_cached_data(self, key: str, fetch_func):
        if cached := self.redis.get(key):
            return json.loads(cached)
        value = await fetch_func()
        self.redis.set(key, json.dumps(value), ex=self.config.CACHE_TTL)
        return value

    async def fetch_ticker_data(self, ticker: str) -> Dict:
        async def fetch():
            ticker_obj = yf.Ticker(ticker)
            history = ticker_obj.history(period="1w", interval="1d")
            return {
                "info": ticker_obj.info,
                "open": float(history["Open"].iloc[-1]),
                "high": float(history["High"].iloc[-1]),
                "low": float(history["Low"].iloc[-1]),
                "close": float(history["Close"].iloc[-1]),
                "volume": float(history["Volume"].iloc[-1]),
                "change": float((history["Close"].iloc[-1] - history["Close"].iloc[0]) / history["Close"].iloc[0])
            }
        return await self.get_cached_data(f"ticker_{ticker}", fetch)

    async def fetch_news(self, ticker: str) -> List[Dict]:
        async def fetch():
            results = self.tavily_client.search(
                query=f"Latest {ticker} news and analysis",
                search_depth="advanced",
                max_results=10
            )
            return [{
                "title": r["title"],
                "content": r["content"],
                "url": r["url"],
                "published_date": r.get("published_date")
            } for r in results["results"]]
        return await self.get_cached_data(f"news_{ticker}", fetch)

    async def analyze_sentiment(self, texts: List[str]) -> Dict:
        combined_text = " ".join(texts)
        blob = TextBlob(combined_text)
        doc = self.nlp(combined_text)
        
        entity_sentiments = {}
        for ent in doc.ents:
            entity_blob = TextBlob(ent.text)
            entity_sentiments[ent.text] = entity_blob.sentiment.polarity

        return {
            "polarity": blob.sentiment.polarity,
            "subjectivity": blob.sentiment.subjectivity,
            "entities": entity_sentiments,
            "confidence": 1 - blob.sentiment.subjectivity
        }

    def generate_prompt(self, content: str) -> str:
        doc = self.nlp(content)
        key_phrases = [sent.text.strip() for sent in doc.sents][:2]
        nouns = [token.text for token in doc if token.pos_ in ["NOUN", "PROPN"]][:5]

        prompt = f"""
        Create a visual scene incorporating these financial concepts:
        Context: {' '.join(key_phrases)}
        Elements: {', '.join(nouns)}

        Describe a striking visual composition that:
        1. Features dramatic lighting, perspective, and depth
        2. Incorporates financial symbols and market indicators
        3. Uses professional color schemes and textures
        4. Creates a powerful mood and atmosphere
        
        Provide only the visual description, focused on artistic elements.
        """

        response = self.openai_client.chat.completions.create(
            model="llama-3.2-3b-preview",
            messages=[
                {"role": "system", "content": "You are an expert art director specializing in financial visualization."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=100,
            temperature=0.7
        )
        return response.choices[0].message.content.strip()

    async def generate_content(self, ticker_data: Dict, sentiment: Dict, news: List[Dict]) -> Dict:
        prompt = f"""
        Write a detailed analysis of {ticker_data['info']['shortName']}:
        
        Market Data:
        - Price: ${ticker_data['close']:.2f}
        - Change: {ticker_data['change']:.2%}
        
        Sentiment:
        - Overall: {sentiment['polarity']:.2f}
        - Confidence: {sentiment['confidence']:.2f}
        
        Recent Headlines:
        {' | '.join(article['title'] for article in news[:3])}
        
        Format in Markdown with technical analysis and outlook.
        """

        response = self.openai_client.chat.completions.create(
            model="llama-3.2-3b-preview",
            messages=[
                {"role": "system", "content": "You are a professional financial analyst."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=2000,
            temperature=0
        )
        
        content = response.choices[0].message.content
        return {
            "content": content,
            "title": f"Analysis: {ticker_data['info']['shortName']}",
            "timestamp": datetime.now().isoformat(),
            "sentiment": sentiment,
            "prompt": self.generate_prompt(content)
        }

    async def save_report(self, category: str, ticker: str, content: Dict):
        output_dir = f"output/{category}/posts"
        os.makedirs(output_dir, exist_ok=True)
        
        filename = f"{ticker}_{datetime.now().strftime('%Y%m%d%H%M%S')}.md"
        path = os.path.join(output_dir, filename)
        
        with open(path, "w") as f:
            f.write(f"# {content['title']}\n\n")
            f.write(f"**Timestamp:** {content['timestamp']}\n\n")
            f.write(f"**Content:**\n\n{content['content']}\n\n")
            f.write("**Sentiment Analysis:**\n\n")
            f.write(f"- Polarity: {content['sentiment']['polarity']:.2f}\n")
            f.write(f"- Confidence: {content['sentiment']['confidence']:.2f}\n\n")
            f.write(f"**Generated Prompt:**\n\n{content['prompt']}\n")

    async def process_ticker(self, category: str, ticker: str):
        try:
            ticker_data = await self.fetch_ticker_data(ticker)
            news = await self.fetch_news(ticker)
            sentiment = await self.analyze_sentiment([article["content"] for article in news])
            content = await self.generate_content(ticker_data, sentiment, news)
            await self.save_report(category, ticker, content)
            await asyncio.sleep(5)  # Rate limiting
        except Exception as e:
            self.logger.error("ticker_processing_failed", ticker=ticker, error=str(e))

    async def run(self):
        try:
            with open("portfolio.json", "r") as f:
                portfolio = json.load(f)
            
            tasks = [
                self.process_ticker(category, ticker)
                for category, tickers in portfolio.items()
                for ticker in tickers
            ]
            
            await asyncio.gather(*tasks, return_exceptions=True)
        except Exception as e:
            self.logger.error("analysis_run_failed", error=str(e))

if __name__ == "__main__":
    analyzer = MarketAnalyzer()
    asyncio.run(analyzer.run())


DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='c:\\Users\\hermi\\OneDrive\\Desktop\\financial_news_reports\\market_analysis\\Lib\\site-packages\\certifi\\cacert.pem'
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'system', 'content': 'You are a professional financial analyst.'}, {'role': 'user', 'content': '\n        Write a detailed analysis of Bitcoin USD:\n        \n        Market Data:\n        - Price: $93621.17\n        - Change: 37.82%\n        \n        Sentiment:\n        - Overall: 0.09\n        - Confidence: 0.58\n        \n        Recent Headlines:\n        Bitcoin USD (BTC-USD) Latest Stock News & Headlines - Yahoo Finance | BTCUSD - Bitcoin - USD News - Barchart.com | Bitcoin USD (BTC-USD) Stock Price, Quote, News & Analysis - Seeking Alpha\n        \n        Format in Markdown with technical analysis and ou

In [ ]:
import os
import json
import logging
import asyncio
import aiohttp
import spacy
import yfinance as yf
import structlog
import numpy as np
import requests
import io
import random
import time
from PIL import Image
from datetime import datetime
from bs4 import BeautifulSoup
from openai import OpenAI
from textblob import TextBlob
from tavily import TavilyClient
from redis import Redis
from pydantic_settings import BaseSettings
from typing import Optional, Dict, List, Any, Tuple

logging.basicConfig(level=logging.DEBUG)

class Settings(BaseSettings):
    GROQ_API_KEY: str
    TAVILY_API_KEY: str
    HF_API_TOKEN: str
    REDIS_URL: str
    DEBUG_LEVEL: str = "DEBUG"
    CACHE_TTL: int = 3600

    class Config:
        env_file = ".env"

class MarketAnalyzer:
    def __init__(self):
        self.config = Settings()
        self.nlp = spacy.load("en_core_web_sm")
        self.redis = Redis.from_url(self.config.REDIS_URL)
        self.logger = structlog.get_logger()
        self.tavily_client = TavilyClient(api_key=self.config.TAVILY_API_KEY)
        self.openai_client = OpenAI(
            base_url="https://api.groq.com/openai/v1",
            api_key=self.config.GROQ_API_KEY,
            max_retries=5,
            timeout=30
        )
        
    async def get_cached_data(self, key: str, fetch_func):
        if cached := self.redis.get(key):
            return json.loads(cached)
        value = await fetch_func()
        self.redis.set(key, json.dumps(value), ex=self.config.CACHE_TTL)
        return value

    async def fetch_ticker_data(self, ticker: str) -> Dict:
        async def fetch():
            ticker_obj = yf.Ticker(ticker)
            history = ticker_obj.history(period="1mo")
            return {
                "info": ticker_obj.info,
                "price": float(history["Close"].iloc[-1]),
                "volume": float(history["Volume"].iloc[-1]),
                "change": float((history["Close"].iloc[-1] - history["Close"].iloc[0]) / history["Close"].iloc[0])
            }
        return await self.get_cached_data(f"ticker_{ticker}", fetch)

    async def fetch_news(self, ticker: str) -> List[Dict]:
        async def fetch():
            results = self.tavily_client.search(
                query=f"Latest {ticker} news and analysis",
                search_depth="advanced",
                max_results=10
            )
            return [{
                "title": r["title"],
                "content": r["content"],
                "url": r["url"],
                "published_date": r.get("published_date")
            } for r in results["results"]]
        return await self.get_cached_data(f"news_{ticker}", fetch)

    async def analyze_sentiment(self, texts: List[str]) -> Dict:
        combined_text = " ".join(texts)
        blob = TextBlob(combined_text)
        doc = self.nlp(combined_text)
        
        entity_sentiments = {}
        for ent in doc.ents:
            entity_blob = TextBlob(ent.text)
            entity_sentiments[ent.text] = entity_blob.sentiment.polarity

        return {
            "polarity": blob.sentiment.polarity,
            "subjectivity": blob.sentiment.subjectivity,
            "entities": entity_sentiments,
            "confidence": 1 - blob.sentiment.subjectivity
        }

    def summarize_with_spacy(self, text: str, max_sentences: int = 3) -> str:
        if isinstance(text, dict):
            text = str(text)
        doc = self.nlp(text)
        sentences = [sent.text.strip() for sent in doc.sents]

        word_freq = {}
        for token in doc:
            if not token.is_stop and not token.is_punct:
                word_freq[token.text] = word_freq.get(token.text, 0) + 1

        sentence_scores = {}
        for sent in sentences:
            score = sum(word_freq.get(word, 0) for word in sent.split())
            sentence_scores[sent] = score

        summary_sentences = sorted(
            sentence_scores.items(), key=lambda x: x[1], reverse=True
        )[:max_sentences]

        return " ".join(sent[0] for sent in summary_sentences)
    
    def extract_nouns(self, text: str) -> List[str]:
        if isinstance(text, dict):
            text = str(text)
        doc = self.nlp(text)
        nouns = [token.text for token in doc if token.pos_ in ["NOUN", "PROPN"]]
        return list(set(nouns))


    def generate_prompt(self, content: str) -> str:
        summary = self.summarize_with_spacy(content, max_sentences=3)
        nouns = self.extract_nouns(summary)
        key_phrases = ", ".join(nouns[:7])

        prompt = f"""
        Create a visual scene incorporating these financial concepts:
        Context: {' '.join(key_phrases)}
        Elements: {', '.join(nouns)}

        Describe a striking visual composition that:
        1. Features dramatic lighting, perspective, and depth
        2. Incorporates financial symbols and market indicators
        3. Uses professional color schemes and textures
        4. Creates a powerful mood and atmosphere
        
        Provide only the visual description, focused on artistic elements.
        """

        response = self.openai_client.chat.completions.create(
            model="llama-3.2-3b-preview",
            messages=[
                {"role": "system", "content": "You are an expert art director specializing in financial visualization."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=80,
            temperature=0.5
        )
        return response.choices[0].message.content.strip()

    def generate_image(
        self,
        prompt: str,
        output_dir: str = ".",
        width: int = 1024,
        height: int = 768,
        num_inference_steps: int = 50,
        guidance_scale: float = 9,
        seed: Optional[int] = None,
        scheduler: str = "heunpp2",
    ) -> Tuple[Image.Image, str]:
        API_URL = "https://api-inference.huggingface.co/models/black-forest-labs/FLUX.1-dev"
        headers = {"Authorization": f"Bearer {self.config.HF_API_TOKEN}"}

        seed = random.randint(0, 2**32 - 1) if seed is None else seed

        payload = {
            "inputs": prompt,
            "parameters": {
                "width": width,
                "height": height,
                "num_inference_steps": num_inference_steps,
                "guidance_scale": guidance_scale,
                "seed": seed,
                "scheduler": scheduler,
            },
        }

        response = requests.post(API_URL, headers=headers, json=payload)

        if response.status_code != 200:
            raise Exception(f"Image generation failed: {response.text}")

        image = Image.open(io.BytesIO(response.content))
        timestamp = int(time.time())
        filename = f"image_{timestamp}.png"
        output_path = os.path.join(output_dir, filename)
        
        os.makedirs(output_dir, exist_ok=True)
        image.save(output_path)

        return image, output_path

    async def generate_content(self, ticker_data: Dict, sentiment: Dict, news: List[Dict]) -> Dict:
        prompt = f"""
        Write a detailed analysis of {ticker_data['info']['shortName']}:
        
        Market Data:
        - Price: ${ticker_data['price']:.2f}
        - Change: {ticker_data['change']:.2%}
        
        Sentiment:
        - Overall: {sentiment['polarity']:.2f}
        - Confidence: {sentiment['confidence']:.2f}
        
        Recent Headlines:
        {' | '.join(article['title'] for article in news[:3])}
        
        Format in Markdown with technical analysis and outlook.
        """

        response = self.openai_client.chat.completions.create(
            model="llama-3.2-3b-preview",
            messages=[
                {"role": "system", "content": "You are a professional financial analyst."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=2000,
            temperature=0
        )
        
        content = response.choices[0].message.content
        return {
            "content": content,
            "title": f"Analysis: {ticker_data['info']['shortName']}",
            "timestamp": datetime.now().isoformat(),
            "sentiment": sentiment,
            "prompt": self.generate_prompt(content)
        }

    async def save_report(self, category: str, ticker: str, content: Dict):
        output_dir = f"output/{category}"
        posts_dir = f"{output_dir}/posts"
        images_dir = f"{output_dir}/images"
        os.makedirs(posts_dir, exist_ok=True)
        os.makedirs(images_dir, exist_ok=True)
        
        try:
            image, image_path = self.generate_image(
                prompt=content['prompt'],
                output_dir=images_dir
            )
            content['image_path'] = image_path
        except Exception as e:
            self.logger.error("image_generation_failed", error=str(e))
            content['image_path'] = None
        
        filename = f"{ticker}_{datetime.now().strftime('%Y%m%d%H%M%S')}.md"
        path = os.path.join(posts_dir, filename)
        
        with open(path, "w") as f:
            f.write(f"# {content['title']}\n\n")
            f.write(f"**Timestamp:** {content['timestamp']}\n\n")
            
            if content.get('image_path'):
                f.write("## Market Analysis Visualization\n\n")
                f.write(f"![Market Analysis for {ticker}]({content['image_path']})\n\n")
            
            f.write(f"**Content:**\n\n{content['content']}\n\n")
            f.write("**Sentiment Analysis:**\n\n")
            f.write(f"- Polarity: {content['sentiment']['polarity']:.2f}\n")
            f.write(f"- Confidence: {content['sentiment']['confidence']:.2f}\n\n")
            f.write(f"**Generated Prompt:**\n\n{content['prompt']}\n")

    async def process_ticker(self, category: str, ticker: str):
        try:
            ticker_data = await self.fetch_ticker_data(ticker)
            news = await self.fetch_news(ticker)
            sentiment = await self.analyze_sentiment([article["content"] for article in news])
            content = await self.generate_content(ticker_data, sentiment, news)
            await self.save_report(category, ticker, content)
            await asyncio.sleep(5)  # Rate limiting
        except Exception as e:
            self.logger.error("ticker_processing_failed", ticker=ticker, error=str(e))

    async def run(self):
        try:
            with open("portfolio.json", "r") as f:
                portfolio = json.load(f)
            
            tasks = [
                self.process_ticker(category, ticker)
                for category, tickers in portfolio.items()
                for ticker in tickers
            ]
            
            await asyncio.gather(*tasks, return_exceptions=True)
        except Exception as e:
            self.logger.error("analysis_run_failed", error=str(e))

if __name__ == "__main__":
    analyzer = MarketAnalyzer()
    asyncio.run(analyzer.run())
